In [5]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('data/longlatdata.csv', names=['City','Latdeg','Latmin','Longdeg','Longmin','Time'])
df.head()

,City,Latdeg,Latmin,Longdeg,Longmin,Time
0,"Aberdeen, Scotland",57,9 N,2,9 W,5:00 p.m.
1,"Adelaide, Australia",34,55 S,138,36 E,2:30 a.m.1
2,"Algiers, Algeria",36,50 N,3,0 E,6:00 p.m.
3,"Amsterdam, Netherlands",52,22 N,4,53 E,6:00 p.m.
4,"Ankara, Turkey",39,55 N,32,55 E,7:00 p.m.


In [9]:
def convert_dm_to_dd(degrees, minutes):
    num, direction = minutes.split(' ')
    dd = float(degrees) + float(num) / 60

    dd = -dd if direction in ['W', 'S'] else dd

    return dd

ndf = pd.DataFrame(df['City'])
ndf['Latitude'] = [convert_dm_to_dd(row[1], row[2]) for i, row in df.iterrows()]
ndf['Longitude'] = [convert_dm_to_dd(row[3], row[4]) for i, row in df.iterrows()]
ndf.head()


,City,Latitude,Longitude
0,"Aberdeen, Scotland",57.150000,-2.150000
1,"Adelaide, Australia",-34.916667,138.600000
2,"Algiers, Algeria",36.833333,3.000000
3,"Amsterdam, Netherlands",52.366667,4.883333
4,"Ankara, Turkey",39.916667,32.916667


In [11]:
from geopy import distance

def get_distance(start, target):
    return distance.distance(start, target)


start_locations = []
target_locations = []
start_latitudes = []
start_longitudes = []
end_latitudes = []
end_longitudes = []
distances = []
for i, row_i in ndf.iterrows():
    start_long_lat = (row_i['Latitude'], row_i['Longitude'])
    for j, row_j in ndf.iterrows():
        target_long_lat = (row_j['Latitude'], row_j['Longitude'])
        d = get_distance(start_long_lat, target_long_lat).km

        distances.append(d)
        start_locations.append(row_i['City'])
        target_locations.append(row_j['City'])

        start_latitudes.append(row_i['Latitude'])
        start_longitudes.append(row_i['Longitude'])

        end_latitudes.append(row_j['Latitude'])
        end_longitudes.append(row_j['Longitude'])
        

distances_df = pd.DataFrame()
distances_df['Start'] = start_locations
distances_df['Target'] = target_locations
distances_df['StartLatitude'] = start_latitudes
distances_df['TargetLatitude'] = end_latitudes
distances_df['StartLongitude'] = start_longitudes
distances_df['TargetLongitude'] = end_longitudes
distances_df['Distance'] = distances
distances_df.head()

,Start,Target,StartLatitude,TargetLatitude,StartLongitude,TargetLongitude,Distance
0,"Aberdeen, Scotland","Aberdeen, Scotland",57.15,57.150000,-2.15,-2.150000,0.000000
1,"Aberdeen, Scotland","Adelaide, Australia",57.15,-34.916667,-2.15,138.600000,16183.676404
2,"Aberdeen, Scotland","Algiers, Algeria",57.15,36.833333,-2.15,3.000000,2290.736724
3,"Aberdeen, Scotland","Amsterdam, Netherlands",57.15,52.366667,-2.15,4.883333,698.243287
4,"Aberdeen, Scotland","Ankara, Turkey",57.15,39.916667,-2.15,32.916667,3164.736857


In [ ]:
distances_df.to_csv('data/distanceslonglat.csv')